In [99]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [100]:
# Load environment variables
# Save OPENAI_API_KEY in the .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [101]:
system_prompt = "You are an assistant that analyzes the contents of a portfolio website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [102]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this portfolio website is as follows; \
please provide a short summary of this website in markdown. \
If it includes CV, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [103]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [104]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [105]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [106]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [107]:
display_summary("https://howardzhan2025.top")

# Summary of Howard (Huaye) Zhan's Portfolio Website

## About
Howard (Huaye) Zhan is currently a student in the AI-Software Engineering Technology program at Centennial College, with a focus on Big Data and Machine Learning. He is seeking a position as a Data Engineer/Machine Learning Engineer in the Greater Toronto Area starting in Spring/Summer 2025. He holds a Bachelor’s degree in Robotics Engineering and Automation from Guangdong University of Technology, where he completed a thesis on Human Pose Estimation Based on Deep Learning. His experience includes working at ZMROBO as a ROS software engineer, developing AI robots and educational courses.

## Projects
Howard has participated in multiple projects, including:
- **Sentiment Analysis on Movie Reviews** using a predictive model with pre-trained BERT.
- **Facial Recognition and Identification** through unsupervised learning.
- **ETL study on Environmental Quality vs. Housing Prices** in Canada.
- **Modeling Traffic Collision Events in Toronto**.
- **Infant Gut Microbial Biomarkers** presentation at IUBDC.
- Secured third place in the **TTC-Hackathon** at Centennial College.

## CV Summary
- **Current Education:** AI-Software Engineer Technology, Centennial College.
- **Previous Education:** Bachelor's in Robotics Engineering and Automation, Guangdong University of Technology.
- **Work Experience:** 
  - ROS software engineer at ZMROBO.
  - Designed educational courses and developed robotic products.

## Personal Interests
In his free time, Howard enjoys jogging, playing frisbee, reading, listening to various music genres, and exploring new interests.

## Blog
Howard has shared insights on various topics in his blog, including AI ethics, MySQL syntax, and neural networks.

--- 

This summary highlights Howard Zhan's educational background, professional experience, projects he has worked on, and personal interests, providing a comprehensive overview of his portfolio.

## Add streaming 

In [108]:
def stream_display(url):
    website = Website(url)
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(website)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [109]:
stream_display("https://howardzhan2025.top")

# Howard (Huaye) Zhan - Portfolio Summary

Howard (Huaye) Zhan is a current student in AI-Software Engineering Technology at Centennial College, focusing on Big Data and Machine Learning. He obtained his Bachelor's degree in Robotics Engineering and Automation from Guangdong University of Technology, where he conducted his thesis on Human Pose Estimation using Deep Learning. 

## Current Interests
- Data Engineer
- Machine Learning Engineer

## Work Experience
- **ZMROBO**: Worked as a ROS software engineer designing AI robots and developing educational courses.

## Projects & Achievements
- Participated in various projects, including:
  - *Sentiment Analysis on Movie Reviews* (Dec 2024)
  - *Facial Recognition and Identification* (Nov 2024)
  - *ETL Study on Environmental Quality and Housing Prices in Canada* (Aug 2024)
  - *Traffic Collision Model for Toronto* (Aug 2024)
  - Presented a paper on **Infant Gut Microbial Biomarkers** at IUBDC 2024.

- Achieved third place in the TTC-Hackathon at Centennial College (Mar 2024).

## Interests
In his free time, Howard enjoys jogging, playing frisbee, reading, and listening to rap and rock music.

## Blog
- Latest topics include AI ethics, MySQL syntax, and a blog post on Generative AI tools.

## CV Summary
- **Education**: 
  - B.Sc. in Robotics Engineering and Automation
  - Currently studying AI software engineering.

- **Skills & Focus Areas**:
  - Big Data
  - Machine Learning
  - ROS (Robot Operating System)
  - Deep Learning

- **Professional Experience**:
  - Developed educational content and advanced robotic technologies at ZMROBO.

Howard is actively seeking opportunities as a Data Engineer or Machine Learning Engineer, aiming to start in Spring/Summer 2025 in the Greater Toronto Area (GTA).